In [1]:
from causalnex.structure.notears import from_pandas
from causalnex.plots import plot_structure, NODE_STYLE, EDGE_STYLE
from IPython.display import Image
import warnings
import pandas as pd

warnings.filterwarnings('ignore')

/home/micky/anaconda3/envs/causal_new/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('../data/cleaned_data.csv')
df.head()

,area_worst,area_mean,area_se,perimeter_worst,perimeter_mean,radius_worst,radius_mean,perimeter_se,texture_worst,texture_mean,diagnosis
0,0.321214,0.259233,0.195146,0.476307,0.389128,0.442429,0.371345,0.263011,0.100865,0.016148,1.0
1,0.300951,0.346851,0.086894,0.373285,0.425815,0.419673,0.444735,0.086051,0.209920,0.188485,1.0
2,0.266958,0.320355,0.116134,0.362429,0.424660,0.396605,0.428760,0.128572,0.256670,0.278187,1.0
3,0.128569,0.140738,0.052182,0.330075,0.319345,0.339598,0.287327,0.173218,0.527809,0.493496,1.0
4,0.255061,0.365362,0.122229,0.378548,0.471170,0.388103,0.470353,0.164698,0.092544,0.116920,1.0


In [3]:
df_train = df.head(int(0.8*len(df)))
df_hold = df.tail(int(0.2*len(df)))

In [4]:
sample_20 = df_train.sample(int(0.2*len(df_train)), random_state=2)
sample_40 = df_train.sample(int(0.4*len(df_train)), random_state=2)
sample_60 = df_train.sample(int(0.6*len(df_train)), random_state=2)
sample_80 = df_train.sample(int(0.8*len(df_train)), random_state=2)

In [5]:
sm = from_pandas(df_train,w_threshold=0.8)
sm_20 = from_pandas(sample_20,w_threshold=0.8)
sm_40 = from_pandas(sample_40,w_threshold=0.8)
sm_60 = from_pandas(sample_60,w_threshold=0.8)
sm_80 = from_pandas(sample_80,w_threshold=0.8)

In [ ]:
viz = plot_structure(
    sm,
    graph_attributes={"scale": "3"},
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK)
Image(viz.draw(format='png'))

In [ ]:
def jaccard_similarity(g, h, th1, th2):
    g.remove_edges_below_threshold(th1)
    h.remove_edges_below_threshold(th2)
    a = g.edges
    b = h.edges
    i = set(a).intersection(b)
    return round(len(i) / (len(a) + len(b) - len(i)),3)